In [33]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import os
import numpy as np
from PIL import Image
import time
import functools
import tensorflow as tf
# import tensorflow.contrib.eager as tfe

from tensorflow.keras.preprocessing import image as kp_image


from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

from matplotlib.dviread import Tfm

In [34]:
import tensorflow as tf

resnet = tf.keras.applications.resnet50.ResNet50(weights='imagenet')
# Content layer where will pull our feature maps
content_layers = ['conv5_block1_1_bn'] 

# Style layer we are interested in
style_layers = ['conv5_block1_1_bn','conv2_block1_2_bn'#,'conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)
# Function to create the style transfer model
def get_model(style_layers, content_layers):
    """Creates a model that outputs intermediate layers of ResNet50.
    
    Args:
        style_layers: List of layer names for style representation.
        content_layers: List of layer names for content representation.
        
    Returns:
        A Keras model that takes image inputs and outputs the specified intermediate layers.
    """
    # Load the pre-trained ResNet50 model
    resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
    resnet.trainable = False
    
    # Get the output layers corresponding to style and content layers
    style_outputs = [resnet.get_layer(name).output for name in style_layers]
    content_outputs = [resnet.get_layer(name).output for name in content_layers]
    
    # Concatenate the outputs for both style and content layers
    model_outputs = style_outputs + content_outputs
    
    # Define the model with ResNet50 input and the selected intermediate layers as outputs
    model = models.Model(resnet.input, model_outputs)
    
    return model



In [35]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [36]:

def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.resnet50.preprocess_input(img)
  return img
  



In [37]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [38]:
def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

In [39]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

In [40]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

In [41]:
def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

In [45]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
content_path = 'input_image.jpg'
style_path = 'style.jpg'

def run_style_transfer(content_path, style_path, model_weights_path):

    # Load the pre-trained model with loaded weights
    model = get_model(style_layers, content_layers)
    model.load_weights(model_weights_path)

    # Freeze all layers to prevent training during style transfer
    for layer in model.layers:
        layer.trainable = False

    style_features, content_features = get_feature_representations(model, content_path, style_path)
    gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
    # Load and process the content image
    init_image = load_and_process_img(content_path)
    init_image = tf.Variable(init_image, dtype=tf.float32)

    # Create a nice config 
    loss_weights = (style_weight, content_weight)
    cfg = {
        'model': model,
        'loss_weights': loss_weights,
        'init_image': init_image,
        'gram_style_features': gram_style_features,
        'content_features': content_features
    }

    # Perform style transfer using the pre-trained model and features (no loss calculation)
    processed_image = model(tf.expand_dims(init_image, axis=0))[0]

    # Deprocess the image back to its original format
    stylized_image = deprocess_img(processed_image.numpy())

    # Display the stylized image
    plt.imshow(stylized_image)
    plt.axis('off')
    plt.show()


In [ ]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

In [65]:
def run_style_transfer(content_features, style_features, model_weights_path):
    # Load the pre-trained model with loaded weights
    model = get_model(style_layers, content_layers)
    model.load_weights(model_weights_path)  # Load model weights from the provided path

    # Define optimizer (you need to define your optimizer, assuming 'optimizer' is an instance of tf.keras.optimizers.Optimizer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    # Initialize the image to be optimized# Get the expected input shape of the model
    # Initialize the image to be optimized with the correct shape
    # Load and preprocess the content image to get its shape
    content_img = load_and_process_img(content_path)
    input_shape = content_img.shape

# Initialize the image to be optimized with the correct shape
    init_image = tf.Variable(tf.zeros(input_shape), trainable=True)

    
 cfg = {
        'model': model,
        'loss_weights': loss_weights,
        'init_image': init_image,
        'gram_style_features': gram_style_features,
        'content_features': content_features
    }
    

    # Main optimization loop
    num_iterations = 1000
    for i in range(num_iterations):
        # Forward pass
        with tf.GradientTape() as tape:
            stylized_image = model(init_image)

            # Calculate loss (replace with your specific loss function)
            content_loss = content_feature_loss(stylized_image, content_features)
            style_loss = style_feature_loss(stylized_image, style_features)
            total_loss = content_loss + style_loss

        # Calculate gradients
        grads = tape.gradient(total_loss, init_image)

        # Update the image
        optimizer.apply_gradients([(grads, init_image)])

        # Print progress
        if i % 100 == 0:
            print("Iteration:", i, "Total loss:", total_loss) 

         grads, all_loss = compute_grads(cfg)
         loss, style_score, content_score = all_loss
         opt.apply_gradients([(grads, init_image)])
         clipped = tf.clip_by_value(init_image, min_vals, max_vals)
         init_image.assign(clipped)
         


            # Use the .numpy() method to get the concrete numpy array
            plot_img = init_image.numpy()
            plot_img = deprocess_img(plot_img)
            imgs.append(plot_img)
            IPython.display.clear_output(wait=True)
            IPython.display.display_png(Image.fromarray(plot_img))
          
        IPython.display.clear_output(wait=True)
        plt.figure(figsize=(14,4))
        for i,img in enumerate(imgs):
            plt.subplot(num_rows,num_cols,i+1)
            plt.imshow(img)
            plt.xticks([])
            plt.yticks([])    

    # # Deprocess the final stylized image
    # final_stylized_image = deprocess_img(stylized_image.numpy())
    # plt.imshow(final_stylized_image)
    # plt.axis('off')
    # plt.show()

style_features, content_features = get_feature_representations(model, content_path, style_path)
run_style_transfer(content_features, style_features, "model_weights.weights.h5")  # Pass the model weights path as a string


NameError: name 'content_feature_loss' is not defined

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

def run_style_transfer(content_features, style_features, model_weights_path):
    # Load the pre-trained model with loaded weights
    model = get_model(style_layers, content_layers)
    model.load_weights(model_weights_path)  # Load model weights from the provided path

    # Define optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    # Initialize the image to be optimized
    init_image = tf.Variable(tf.random.normal(content_features.shape), trainable=True)

    # Define loss weights
    loss_weights = (1e4, 1e-2)  # Adjust these weights as needed

    # Precompute gram style features
    gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

    # Main optimization loop
    num_iterations = 1000
    for i in range(num_iterations):
        # Compute gradients and loss
        grads, (total_loss, style_loss, content_loss) = compute_grads({
            'model': model,
            'loss_weights': loss_weights,
            'init_image': init_image,
            'gram_style_features': gram_style_features,
            'content_features': content_features
        })

        # Update the image
        optimizer.apply_gradients([(grads, init_image)])

        # Print progress
        if i % 100 == 0:
            print("Iteration:", i, "Total loss:", total_loss.numpy())

            # Deprocess the stylized image and display
            stylized_img_np = deprocess_img(init_image.numpy())
            plt.imshow(stylized_img_np)
            plt.axis('off')
            plt.show()

def compute_loss(cfg):
    model = cfg['model']
    loss_weights = cfg['loss_weights']
    init_image = cfg['init_image']
    gram_style_features = cfg['gram_style_features']
    content_features = cfg['content_features']

    style_weight, content_weight = loss_weights

    # Feed our init image through our model. This will give us the content and 
    # style representations at our desired layers. Since we're using eager
    # our model is callable just like any other function!
    model_outputs = model(init_image)

    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]

    style_score = 0
    content_score = 0

    # Accumulate style losses from all layers
    # Here, we equally weight each contribution of each loss layer
    weight_per_style_layer = 1.0 / float(num_style_layers)
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)

    # Accumulate content losses from all layers 
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += weight_per_content_layer * get_content_loss(comb_content[0], target_content)

    style_score *= style_weight
    content_score *= content_weight

    # Get total loss
    loss = style_score + content_score 
    return loss, style_score, content_score

def compute_grads(cfg):
    with tf.GradientTape() as tape: 
        all_loss = compute_loss(cfg)
    # Compute gradients wrt input image
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss

# Define your content and style features
content_features = ...
style_features = ...

# Run style transfer
run_style_transfer(content_features, style_features, "model_weights.weights.h5")
